In [1]:
import sys
sys.path.append('./libs')
import util_modelo_final
import importlib

importlib.reload(util_modelo_final)

tuned_parameters = [{'keras__batch_size': [2**10], 'keras__epochs': [2**10], 'keras__optimizer': ['Adam'], 'keras__learn_rate': [0.01], 'keras__momentum': [0], 'keras__init_mode': ['uniform'], 'keras__activation': ['relu'], 'keras__activation2': ['softmax'], 'keras__dropout_rate': [0.0], 'keras__weight_constraint': [1], 'keras__neurons': [10]}]
tesis2 = util_modelo_final.Tesis2(carpeta_base="real",verbose=100,tuned_parameters=tuned_parameters)

cantidad_especies = 30
#cantidad_especies = 3
cantidad_transcritos = 4028
#cantidad_transcritos = 400

Using TensorFlow backend.


In [2]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, dump

def obtener_especies():
    query = "SELECT * FROM (SELECT CONVERT(@row_number:=@row_number + 1, UNSIGNED) AS orden, m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie, (SELECT @row_number:=0) AS rn ORDER BY m.id_especie) a ORDER BY 1 LIMIT " + str(cantidad_especies)
    return util_bd.resultados_query(query)
        
def generar_fasta_especie(tesis2, row_especie):
    os.mkdir(tesis2.carpeta_data() + "/clase_" + str(row_especie[0]))
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, tesis2.carpeta_data() + "/clase_" + str(row_especie[0]) + "/lncRNA.fa")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, tesis2.carpeta_data() + "/clase_" + str(row_especie[0]) + "/PCT.fa")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + str(row_especie[1]) + " ORDER BY f.cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, tesis2.carpeta_data() + "/clase_" + str(row_especie[0]) + "/CDS.fa")

if os.path.isdir(tesis2.carpeta_data()):
    shutil.rmtree(tesis2.carpeta_data())
os.mkdir(tesis2.carpeta_data())

%time dump(obtener_especies(), tesis2.carpeta_data() + "/info_clases.bin")
%time Parallel(n_jobs=tesis2.n_jobs, verbose=tesis2.verbose)(delayed(generar_fasta_especie)(tesis2, row_especie) for row_especie in obtener_especies())

print("Proceso finalizado")

CPU times: user 5.01 ms, sys: 135 µs, total: 5.14 ms
Wall time: 98.1 ms
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | e

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

%time tesis2.generar_modelo_final_keras()

*************************************************
*************** Generando llaves ****************
*************************************************
*************************************************
***************** Armando folds *****************
*************************************************
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: D

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.8min


/home/jose/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  7.9min
[Paralle

[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed: 91.1min
Memmapping (shape=(233624,), dtype=int64) to new file /dev/shm/joblib_memmapping_folder_23178_8487916427/23178-139709879909176-f3954044fee14609bcd74b54718e09bf.pkl
Pickling array (shape=(8056,), dtype=int64).
Memmapping (shape=(233624,), dtype=int64) to new file /dev/shm/joblib_memmapping_folder_23178_8487916427/23178-139709879909176-7bdc41ef853a485a9d3fd79e4a0caf09.pkl
Pickling array (shape=(8056,), dtype=int64).
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 109.0min
Memmapping (shape=(233624,), dtype=int64) to new file /dev/shm/joblib_memmapping_folder_23178_8487916427/23178-139709879909176-42267d57a4c4492f81d092290660a5b1.pkl
Pickling array (shape=(8056,), dtype=int64).
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed: 109.1min
Memmapping (shape=(233624,), dtype=int64) to new file /dev/shm/joblib_memmapping_folder_23178_8487916427/23178-139709879909176-472cfa6a90eb47889bf58b71a61c8474.pkl
Pickling array (shape

{'mean_fit_time': array([1016.86016982]),
 'std_fit_time': array([72.32392445]),
 'mean_score_time': array([2.01997584]),
 'std_score_time': array([0.2567279]),
 'param_keras__activation': masked_array(data=['relu'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_keras__activation2': masked_array(data=['softmax'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_keras__batch_size': masked_array(data=[1024],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_keras__dropout_rate': masked_array(data=[0.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_keras__epochs': masked_array(data=[1024],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_keras__init_mode': masked_array(data=['uniform'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_keras__learn_rate'

{'keras__activation': 'relu',
 'keras__activation2': 'softmax',
 'keras__batch_size': 1024,
 'keras__dropout_rate': 0.0,
 'keras__epochs': 1024,
 'keras__init_mode': 'uniform',
 'keras__learn_rate': 0.01,
 'keras__momentum': 0,
 'keras__neurons': 10,
 'keras__optimizer': 'Adam',
 'keras__weight_constraint': 1}

{'accuracy': 0.877979145978153,
 'precision': 0.8351676561929974,
 'recall': 0.9447120158887785}

CPU times: user 17min 9s, sys: 56.5 s, total: 18min 6s
Wall time: 7h 52min 58s
